## Container vs Flat Sequences, Mutable vs Immutable Sequences

**Container sequences** 
- can hold items of different types, including nested containers. Some examples: `list`, `tuple`, and `collections.deque`.
- holds references to the objects it contains, which may be of any type


**Flat sequences** 
- hold items of one simple type. Some examples: `str`, `bytes`, and `array.array`.
- stores the value of its contents in its own memory space, not as distinct Python objects.
- flat sequences are more compact than container sequences, but they are limited to holding primitive machine values like bytes, integers, and floats.

*The flat sequences are more compact, faster, and easier to use, but are limited to storing atomic data such as numbers, characters, and bytes. Container sequences are more flexible, but may surprise you when they hold mutable objects, so you need to be careful to use them correctly with nested data structures.*

### Another way of grouping sequence types is by mutability:

- **Mutable sequences**, e.g. `list`, `bytearray`, `array.array`, and `collections.deque`. Mutable sequences inherit all methods from immuta‐
ble sequences
- **Immutable sequences**, e.g. `tuple`, `str`, and `bytes`.


In [5]:
a = []
a.append(1)
a.append('2')
a.append([2.5, 3.2])
a

[1, '2', [2.5, 3.2]]

## List Comprehensions and Generator Expressions

A list comprehension is more explicit. *Its goal is always to build a new list. If you do not do anything else with the produced list, don't use this syntax. Also, keep it short for readability*

### Local Scope Within Comprehensions and Generator Expressions
Variables assigned with the “Walrus operator” `:=` remain accessible after
those comprehensions or expressions return—unlike local variables in a function.

In [10]:
x = 'ABC'
codes = [last := ord(c) for c in x]
print(f"{codes = }")
print(f"{last = }")

codes = [65, 66, 67]
last = 67


### Listcomps versus map and filter
Listcomps do everything the map and filter functions do, without the contortions of the functionally challenged Python `lambda`. And listcomps are not slower than map and filter

In [18]:
symbols = '$¢£¥€¤'
beyond_ascii = [ord(s) for s in symbols if ord(s) > 127]
beyond_ascii_filter_map = list(filter(lambda c: c > 127, map(ord, symbols)))
print(f"{beyond_ascii == beyond_ascii_filter_map = }")

beyond_ascii == beyond_ascii_filter_map = True


### Cartesian product using a list comprehension
For example, imagine you need to produce a list of T-shirts available in two colors and three sizes. The following code shows how to produce that list using a listcomp. The result has six items

In [1]:
colors = ['black', 'white']
sizes = ['S', 'M', 'L']
tshirts = [(color, size) for color in colors 
                         for size in sizes]
tshirts

[('black', 'S'),
 ('black', 'M'),
 ('black', 'L'),
 ('white', 'S'),
 ('white', 'M'),
 ('white', 'L')]

get items arranged by size, then color,

In [2]:
tshirts = [(color, size) for size in sizes
                         for color in colors]
tshirts

[('black', 'S'),
 ('white', 'S'),
 ('black', 'M'),
 ('white', 'M'),
 ('black', 'L'),
 ('white', 'L')]

### Generator Expressions
*To initialize tuples, arrays, and other types of sequences, you could also start from a listcomp, but a genexp (generator expression) saves memory because it yields items one by one using the iterator protocol instead of building a whole list just to feed
another constructor*. Genexps use the same syntax as listcomps, but are enclosed in parentheses rather than brackets.

In [3]:
symbols = '$¢£¥€¤'
tuple(ord(symbol) for symbol in symbols)

(36, 162, 163, 165, 8364, 164)

In [5]:
import array
array.array('I', (ord(symbol) for symbol in symbols))

array('I', [36, 162, 163, 165, 8364, 164])

Cartesian product using a genexp to print out a roster of T-shirts of two colors in three sizes. Here the six-item list of T-shirts is never built in memory: the generator expression feeds the for loop producing one item at a time.

In [7]:
colors = ['black', 'white']
sizes = ['S', 'M', 'L']
for tshirt in (f'({c}, {s})' for c in colors 
                          for s in sizes):
    print(tshirt)

(black, S)
(black, M)
(black, L)
(white, S)
(white, M)
(white, L)


## Tuples Are Not Just Immutable Lists
Tuples can be used as immutable lists, and also as records with no field names.

### Tuples as Records
When using a tuple as a collection of fields, the number of items is usually fixed and their order is always important

In [8]:
lax_coordinates = (33.9425, -118.408056)
city, year, pop, chg, area = ('Tokyo', 2003, 32_450, 0.66, 8014)
traveler_ids = [('USA', '31195855'), ('BRA', 'CE342567'), ('ESP', 'XDA205856')]
for passport in sorted(traveler_ids):
    print('%s/%s' % passport)
for country, _ in traveler_ids:
    print(country)

BRA/CE342567
ESP/XDA205856
USA/31195855
USA
BRA
ESP


### Tuples as Immutable Lists

The Python interpreter and standard library make extensive use of tuples as immutable lists, and so should you. This brings two key benefits:
- Clarity: When you see a `tuple` in code, you know *its length will never change*.
- Performance: A `tuple` uses *less memory* than a `list` of the same length, and it allows Python to do some optimizations

However, immutability of a tuple only applies to the references contained in it. References in a tuple cannot be deleted or replaced. But if one of those references points to a mutable object, and that object is changed, then the value of the tuple changes

<img src="../images/1.png" style="width: 40%;">


In [8]:
a = (10, 'alpha', [1, 2])
a

(10, 'alpha', [1, 2])

In [9]:
a[0] = 11

TypeError: 'tuple' object does not support item assignment

In [10]:
a[-1].append(99)
a

(10, 'alpha', [1, 2, 99])

*Hence, tuples with mutable items can be a source of bugs*.  
If you want to determine explicitly if a tuple (or any object) has a fixed value, you can use the hash built-in to create a fixed function like this

In [2]:
def fixed(o):
    try:
        hash(o)
    except TypeError as e:
        print(e)
        return False
    return True

tf = (10, 'alpha', (1, 2))
tm = (10, 'alpha', [1, 2])

print(f"{fixed(tf) = }")
print(f"{fixed(tm) = }")


fixed(tf) = True
unhashable type: 'list'
fixed(tm) = False


Despite this caveat, tuples are widely used as immutable lists.
- To evaluate a tuple literal, the Python compiler generates bytecode for a tuple constant in one operation; but for a list literal, the generated bytecode pushes each element as a separate constant to the data stack, and then builds the list.
- Given a tuple t, `tuple(t)` simply returns a reference to the same t. There’s no need to copy. In contrast, given a list l, the `list(l)` constructor must create a new copy of l.
- Because of its fixed length, a tuple instance is allocated the exact memory space it needs. Instances of list, on the other hand, are allocated with room to spare, to amortize the cost of future appends.
- The references to the items in a tuple are stored in an array in the tuple struct, while a list holds a pointer to an array of references stored elsewhere. The indirection is necessary because when a list grows beyond the space currently allocated, Python needs to reallocate the array of references to make room. The extra indirection makes CPU caches less effective.

### Comparing Tuple and List Methods

When using a tuple as an immutable variation of list, it is good to know how similar their APIs are. Tuple supports all list methods that do not involve adding or removing items, with one exception — tuple lacks the `__reversed__` method.

<img src="../images/5.png" style="width: 40%;">

## Unpacking Sequences and Iterables

Unpacking is important because it avoids unnecessary and error-prone use of indexes to extract elements from sequences.  
The most visible form of unpacking is parallel assignment;

In [3]:
lax_coordinates = (33.9425, -118.408056)
latitude, longitude = lax_coordinates # unpacking
print(latitude, longitude)

33.9425 -118.408056


### Using `*` to Grab Excess Items

Another example of unpacking is prefixing an argument with `*` when calling a function:

In [5]:
t = (20, 8)
quotient, remainder = divmod(*t)
print(f"{quotient = }, {remainder = }")

quotient = 2, remainder = 4


### Unpacking with `*` in Function Calls and Sequence Literals

In function calls, we can use `*` multiple times:

In [8]:
def fun(a, b, c, d, *rest):
    return a, b, c, d, rest

fun(*[1, 2], 3, *range(4, 7))

(1, 2, 3, 4, (5, 6))

The `*` can also be used when defining `list`, `tuple`, or `set` literals

In [9]:
print(*range(4), 4)
print([*range(4), 4])
print({*range(4), 4, *(5, 6, 7)})

0 1 2 3 4
[0, 1, 2, 3, 4]
{0, 1, 2, 3, 4, 5, 6, 7}


### Nested Unpacking

In [11]:
metro_areas = [
    ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),
    ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
    ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
    ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
    ('São Paulo', 'BR', 19.649, (-23.547778, -46.635833)),
]

def main():
    print(f'{"":15} | {"latitude":>9} | {"longitude":>9}')
    for name, _, _, (lat, lon) in metro_areas:  # unpacking here
        if lon <= 0:
            print(f'{name:15} | {lat:9.4f} | {lon:9.4f}')

main()

                |  latitude | longitude
Mexico City     |   19.4333 |  -99.1333
New York-Newark |   40.8086 |  -74.0204
São Paulo       |  -23.5478 |  -46.6358


## Pattern Matching with Sequences
The most visible new feature in Python 3.10 is pattern matching with the `match/case` statement, which uses destructuring — a more advanced form of unpacking.
```python
def handle_command(self, message):
    match message:  # message can be a list of different patterns
        case ['BEEPER', frequency, times]:
            self.beep(times, frequency)
        case ['NECK', angle]:
            self.rotate_neck(angle)
        case ['LED', ident, intensity]:
            self.leds[ident].set_brightness(ident, intensity)
        case ['LED', ident, red, green, blue]:
            self.leds[ident].set_color(ident, red, green, blue)
        case _:
            raise InvalidCommand(message)
```

A sequence pattern can match instances of most actual or virtual subclasses of `collections.abc.Sequence` (`list`, `tuple`, `memoryview`, `range`, `array.array`, `collections.deque`), with the exception of `str`, `bytes`, and `bytearray`.

<img src="../images/6.png" style="width: 80%;">

## Slicing
A common feature of `list`, `tuple`, `str`, and all sequence types in Python is the support of slicing operations, which are more powerful than most people realize

### Why Slices and Ranges Exclude the Last Item
- It’s easy to see the length of a slice or range when only the stop position is given: `range(3)` and `my_list[:3]` both produce three items.
- It’s easy to compute the length of a slice or range when start and stop are given: just subtract `stop - start`.
- It’s easy to split a sequence in two parts at any index x, without overlapping: simply get `my_list[:x]` and `my_list[x:]`.

In [12]:
l = [10, 20, 30, 40, 50, 60]
print(f"{l[:2] = }")
print(f"{l[2:] = }")

l[:2] = [10, 20]
l[2:] = [30, 40, 50, 60]


### Slice Objects
This is no secret, but worth repeating just in case: `s[a:b:c]` can be used to specify a stride or step `c`, causing the resulting slice to skip items.

In [17]:
s = 'bicycle'
print(f"{s[::3] = }")
print(f"{s[::-1] = }")

s[::3] = 'bye'
s[::-1] = 'elcycib'


In [19]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]
    
deck = FrenchDeck()
deck[12::13]

[Card(rank='A', suit='spades'),
 Card(rank='A', suit='diamonds'),
 Card(rank='A', suit='clubs'),
 Card(rank='A', suit='hearts')]

To evaluate the expression `seq[start:stop:step]`, Python calls `seq.__getitem__(slice(start, stop, step))`

### Multidimensional Slicing and Ellipsis
The `[]` operator can also take multiple indexes or slices separated by commas. The `__getitem__` and `__setitem__` special methods that handle the `[]` operator simply receive the indices in `a[i, j]` as a tuple. In other words, to evaluate `a[i, j]`, Python calls `a.__getitem__((i, j))`

This is used, for instance, in the external NumPy package, where items of a two dimensional numpy.ndarray can be fetched using the syntax `a[i, j]` and a two-dimensional slice obtained with an expression like `a[m:n, k:l]`.

In [29]:
import numpy as np

a = np.arange(12)
a.shape = (3, 4)
print(f"{a = }")
print(f"{a[2, 1] = }")
print(f"{a[2, 1:4] = }")

a = array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])
a[2, 1] = 9
a[2, 1:4] = array([ 9, 10, 11])


The ellipsis—written with three full stops (`...`) is an alias to the Ellipsis object, the single instance of the ellipsis class. As such, it can be passed as an argument to functions and as part of a slice specification, as in `f(a, ..., z)` or `a[i:...]`

In [39]:
import numpy as np

arr = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

# Using ... to slice the array
print(arr)
arr[..., 0]  # row whatever, col 0

[[1 2 3]
 [4 5 6]
 [7 8 9]]


array([1, 4, 7])

### Assigning to Slices
Mutable sequences can be grafted, excised, and otherwise modified in place using slice notation on the lefthand side of an assignment statement or as the target of a `del` statement

In [43]:
l = list(range(10))
print(l)
l[2:5] = [20, 30]
print(l)
del l[5:7]
print(l)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 1, 20, 30, 5, 6, 7, 8, 9]
[0, 1, 20, 30, 5, 8, 9]


## Using `+` and `*` with Sequences
Both `+` and `*` always create a new object, and never change their operands.

In [47]:
l = [1, 2, 3]
print(f"{l * 5 = }")
[1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3]
print(f"{5 * 'abcd' = }")

l * 5 = [1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3]
5 * 'abcd' = 'abcdabcdabcdabcdabcd'


### Building Lists of Lists

In [52]:
# A list with three lists of length 3 can represent a tic-tac-toe board
board = [['_'] * 3 for i in range(3)]
print(board)
board[1][2] = 'X'
print(board)

[['_', '_', '_'], ['_', '_', '_'], ['_', '_', '_']]
[['_', '_', '_'], ['_', '_', 'X'], ['_', '_', '_']]


In [53]:
# A list with three references to the same list is useless
weird_board = [['_'] * 3] * 3
print(f"{weird_board = }")
weird_board[1][2] = 'O'
print(f"{weird_board = }")

weird_board = [['_', '_', '_'], ['_', '_', '_'], ['_', '_', '_']]
weird_board = [['_', '_', 'O'], ['_', '_', 'O'], ['_', '_', 'O']]


### Augmented Assignment with Sequences
The special method that makes `+=` work is `__iadd__` (for “in-place addition”). However, when `a` does not implement `__iadd__`, the expression `a += b` has the same effect as `a = a + b`: the expression `a + b` is evaluated first, producing a new object, which is then bound to `a`. In other words, the identity of the object bound to a may or may not change, depending on the availability of
`__iadd__`. Everything is similar to `*=` with `__imul__`  

Example of `*=` with a mutable sequence and then an immutable one

In [55]:
# identity of l is not changed since l is a mutable list
l = [1, 2, 3]
id1 = id(l)
l *= 2
assert id(l) == id1

In [58]:
# After multiplication, a new tuple was created
t = (1, 2, 3)
id1 = id(t)
t *= 2
assert id(t) != id1

### A += Assignment Puzzler
Below is a very interesting problem

In [63]:
t = (1, 2, [30, 40])
print(t)
t[2] += [50, 60]

(1, 2, [30, 40])


TypeError: 'tuple' object does not support item assignment

In [64]:
t

(1, 2, [30, 40, 50, 60])

In [68]:
import dis
dis.dis('s[a] += b')

  1           0 LOAD_NAME                0 (s)
              2 LOAD_NAME                1 (a)
              4 DUP_TOP_TWO
              6 BINARY_SUBSCR
              8 LOAD_NAME                2 (b)
             10 INPLACE_ADD
             12 ROT_THREE
             14 STORE_SUBSCR
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE


Lessons:
- Avoid putting mutable items in tuples.
- Augmented assignment is not an atomic operation—we just saw it throwing an exception after doing part of its job.
- Inspecting Python bytecode is not too difficult, and can be helpful to see what is going on under the hood.

## `list.sort` vs the `sorted` built-in

The `list.sort` method sorts a list in place—that is, without making a copy. It returns `None` to remind us that it changes the receiver and does not create a new list. 

*Functions or methods that change an object inplace should return `None` to make it clear to the caller that the receiver was changed, and no new object was created.*

In contrast, the built-in function `sorted` creates a new list and returns it. It accepts any iterable object as an argument, including immutable sequences and generators.

Both `list.sort` and `sorted` take two optional, keyword-only arguments:
- `reverse`: If True, the items are returned in descending order (i.e., by reversing the comparison of the items). The default is `False`
- `key`: A one-argument function that will be applied to each item to produce its sorting key.

In [71]:
fruits = ['grape', 'raspberry', 'apple', 'banana']
fruits = ['grape', 'raspberry', 'apple', 'banana']
print(f"sorted(fruits): {sorted(fruits)}")  
print(f"fruits: {fruits}")  
print(f"sorted(fruits, reverse=True): {sorted(fruits, reverse=True)}")
print(f"sorted(fruits, key=len): {sorted(fruits, key=len)}")   
print(f"sorted(fruits, key=len, reverse=True): {sorted(fruits, key=len, reverse=True)}")  
print(f"fruits: {fruits}")

sorted(fruits): ['apple', 'banana', 'grape', 'raspberry']
fruits: ['grape', 'raspberry', 'apple', 'banana']
sorted(fruits, reverse=True): ['raspberry', 'grape', 'banana', 'apple']
sorted(fruits, key=len): ['grape', 'apple', 'banana', 'raspberry']
sorted(fruits, key=len, reverse=True): ['raspberry', 'banana', 'grape', 'apple']
fruits: ['grape', 'raspberry', 'apple', 'banana']


In [72]:
fruits.sort()
fruits  # ['apple', 'banana', 'grape', 'raspberry']

['apple', 'banana', 'grape', 'raspberry']

## When a List is Not the Answer

The `list` type is flexible and easy to use, but depending on specific requirements,
there are better options. For example, 
- `array`: An `array` saves a lot of memory when you need to *handle millions of floating-point values*. 
- `deque`: If you are *constantly adding and removing items from opposite ends* of a list, it’s good to know that a `deque` (double-ended queue) is a more efficient FIFO data structure.
- `set`: If your code *frequently checks whether an item is present in a collection* (e.g., `item` in `my_collection`), consider using a `set` for `my_collection`, especially if it holds a large number of items. Sets are optimized for fast membership checking. They are also iterable, but they are not sequences because the ordering of set items is unspecified. The arrangement you see when you print a `set` is not due to any internal sorting but rather the result of how Python stores the elements for efficient access. This storage method is based on the hash values of the elements and can vary between different runs of a program

In [4]:
a = {1,2,3}
print(f"{a = }")
a.add(0)  # will be added to the front
print(f"{a = }")

a = {1, 2, 3}
a = {0, 1, 2, 3}


In [5]:
a.add(4)  # will be added to the back
print(f"{a = }")

a = {0, 1, 2, 3, 4}


### Array

If a list only contains numbers, an `array.array` is a more efficient replacement. Arrays support all mutable sequence operations (including `.pop`, `.insert`, and `.extend`), as well as additional methods for fast loading and saving, such as `.frombytes` and `.tofile`.

In [8]:
from array import array
from random import random

# Create an array of 'd' type (double) with 10 million random float values
floats = array('d', (random() for i in range(10**7)))
# Access the last element of the array
floats[-1]
# Open a binary file 'floats.bin' in write mode
fp = open('floats.bin', 'wb')
# Write the contents of the 'floats' array to the binary file
floats.tofile(fp)
# Close the file
fp.close()
# Create an empty array of 'd' type
floats2 = array('d')
# Open the binary file 'floats.bin' in read mode
fp = open('floats.bin', 'rb')
# Read the contents of the binary file into the 'floats2' array
floats2.fromfile(fp, 10**7)
# Close the file
fp.close()
# Access the last element of the 'floats2' array
floats2[-1]
# Check if the 'floats2' array is equal to the 'floats' array
floats2 == floats
# remove the file
import os
os.remove("floats.bin")

### Memory Views
A memoryview is essentially a generalized NumPy array structure in Python itself (without the math). It allows you to share memory between data-structures (things like PIL images, SQLite databases, NumPy arrays, etc.) without first copying. This is very important for large data sets.

In [14]:
from array import array

octets = array('B', range(6))  # Build array of 6 bytes (typecode 'B').
m1 = memoryview(octets)  # Build memoryview from that array, then export it as a list.
m2 = m1.cast('B', [2, 3])  # Build new memoryview from that previous one, but with 2 rows and 3 columns.
m2.tolist()

[[0, 1, 2], [3, 4, 5]]

### NumPy
For advanced array and matrix operations, NumPy is the reason why Python became mainstream in scientific computing applications. NumPy implements multidimensional, homogeneous arrays and matrix types that hold not only numbers but also user-defined records, and provides efficient element-wise operations.

Scipy is written on top of Numpy offering many scientific computing algorithms from linear algebra, numerical calculus, and statistics.

Most NumPy and SciPy functions are implemented in C or C++, and can leverage all CPU cores because they release Python’s GIL (Global Interpreter Lock).  

Other awesome Python frameworks written on top of Numpy and Scipy:
- `Pandas` implements efficient array types that can hold non-numeric data and provides import/export functions for many different formats, like .csv, .xls, SQL dumps, HDF5, etc
- Scikit-learn, currently the most widely used Machine Learning toolset
- The Dask project supports parallelizing NumPy, Pandas, and scikit-learn processing across clusters of machines

### Deques and Other Queues

The `.append` and `.pop` methods make a list usable as a stack or a queue (if you use `.append` and `.pop(0)`, you get FIFO behavior). But inserting and removing from the head of a list (the 0-index end) is costly because the entire list must be shifted in memory.

The class collections.deque is a thread-safe double-ended queue designed for fast inserting and removing from both ends.

`queue` can also be bounded: If a bounded deque is full, when you add
a new item, it discards an item from the opposite end

In [15]:
from collections import deque

dq = deque(range(10), maxlen=10)
print(dq)
dq.rotate(3)
print(dq)
dq.appendleft(-1)
print(dq)
dq.extend([11, 22, 33])
print(dq)

deque([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], maxlen=10)
deque([7, 8, 9, 0, 1, 2, 3, 4, 5, 6], maxlen=10)
deque([-1, 7, 8, 9, 0, 1, 2, 3, 4, 5], maxlen=10)
deque([9, 0, 1, 2, 3, 4, 5, 11, 22, 33], maxlen=10)


Besides deque, other Python standard library packages implement queues:
- `queue`: This provides the synchronized (i.e., thread-safe) classes `SimpleQueue`, `Queue`, `LifoQueue`, and `PriorityQueue`. These can be used for safe communication between threads.
- `multiprocessing`: Implements its own unbounded `SimpleQueue` and bounded `Queue`, very similar to those in the `queue` package, but designed for interprocess communication.
- `asyncio`: Provides `Queue`, `LifoQueue`, `PriorityQueue`, and `JoinableQueue` with APIs inspired by the classes in the `queue` and multiprocessing modules, but adapted for managing tasks in asynchronous programming.
- `heapq`: In contrast to the previous three modules, `heapq` does not implement a `queue` class, but provides functions like `heappush` and `heappop` that let you use a mutable sequence as a heap queue or priority queue.